<h1>Pre-processing and Labelling of Dataset</h1>

In [2]:

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the image directory and image size
image_dir = 'D:\Conv-Capsnet\Dataset'
image_size = (150, 150)

# Function to load and preprocess images
def load_and_preprocess_data(image_dir, image_size):
    data = []
    labels = []
    classes = ['covid', 'pneumonia', 'normal']
    
    for class_label, class_name in enumerate(classes):
        class_dir = os.path.join(image_dir, class_name)
        for img_filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_filename)
            try:
                # print("Loading image:", img_path)  # Print the image path for debugging
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image as grayscale
                
                if img is None:
                    print(f"Error loading image: {img_path}")
                    continue

                img = cv2.resize(img, image_size)  # Resize image to 150x150
                img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
                data.append(img)
                labels.append(class_label)
            except Exception as e:
                print(f"Error processing image: {img_path} - {str(e)}")

    # Convert lists to numpy arrays
    data = np.array(data)
    labels = np.array(labels)
    
    # Convert labels to one-hot encoded vectors
    labels = to_categorical(labels, num_classes=len(classes))
    
    return data, labels
    
# Load and preprocess the data
data, labels = load_and_preprocess_data(image_dir, image_size)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Print the shape of the training and testing data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (11364, 150, 150)
y_train shape: (11364, 3)
X_test shape: (3789, 150, 150)
y_test shape: (3789, 3)


<h1> Implmentation of the propsed model - ConvCapsnet </h1>

In [3]:
import tensorflow as tf
import numpy as np
from keras import models, layers

In [4]:
# Placeholder for input image
X = tf.keras.layers.Input(shape=(150, 150, 1), name="X", dtype=tf.float32)

In [5]:
# Define the convolutional parameters
conv1_parameters = {
    "filters": 16,
    "kernel_size": (5, 5),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}
conv2_parameters = {
    "filters": 32,
    "kernel_size": (5, 5),
    "strides": (2, 2),
    "padding": "same",
    "activation": tf.nn.relu
}

conv3_parameters = {
    "filters": 64,
    "kernel_size": (5, 5),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}

conv4_parameters = {
    "filters": 128,
    "kernel_size": (9, 9),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}


In [6]:

# Create the four convolutional layers

conv1 = layers.Conv2D(name="conv1", **conv1_parameters)(X)
conv2 = layers.Conv2D(name="conv2", **conv2_parameters)(conv1)
conv3 = layers.Conv2D(name="conv3", **conv3_parameters)(conv2)
conv4 = layers.Conv2D(name="conv4", **conv4_parameters)(conv3)


In [7]:
def squash(s, ax=-1, epsilon=1e-7, name=None): 
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=ax, keepdims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector


In [8]:
# Primary capsule layer 

primary_capsule_input = tf.reshape(conv4, [-1, 1152 , 8], name="primary_capsule_input")
primary_capsule_output = squash(primary_capsule_input, name="primary_capsule_output")

In [9]:
# Secondary Capsule layer 

# Initializing weight matrix 

W_init = tf.random.normal(shape=(1, 1152, 3, 16, 8), stddev=0.01, name="W_init", dtype=tf.float32)
W = tf.Variable(W_init, name="W")

# Tile the weight matrix 

W_tiled = tf.tile(W, [tf.shape(X)[0], 1, 1, 1, 1], name="W_tiled")

In [10]:
# Expand output of primary capsule (batchsize x 1152 x 8) to (batchsize x 1152 x 3 x 8 x 1) 

primary_capsule_output_expanded = tf.expand_dims(primary_capsule_output, -1, name="primar_capsule_output_expanded")
primary_capsule_output_tile = tf.expand_dims(primary_capsule_output_expanded, 2, name="primary_capsule_output_tile")
primary_capsule_output_tiled = tf.tile(primary_capsule_output_tile, [1, 1, 3, 1, 1], name="primary_capsule_output_tiled")

In [11]:
# Secondary Capsule Prediction Vector uj|i = Wij * uij for all i, j 

secondary_capsule_predicted = tf.matmul(W_tiled, primary_capsule_output_tiled, name="secondary_capsule_prediction")

In [12]:
secondary_capsule_predicted

<KerasTensor: shape=(None, 1152, 3, 16, 1) dtype=float32 (created by layer 'tf.linalg.matmul')>

In [13]:
# Dynamic Routing 

# Initialize bias bij to zero 

raw_weights_round1 = tf.zeros([tf.shape(X)[0], 1152, 3, 1, 1], dtype=tf.float32, name="raw_weights_round1")

# Round 1 

# Coupling Coeffecient Cij = softmax(bij) 

routing_weights_round1 = tf.nn.softmax(raw_weights_round1, axis=2, name="routing_weights_round1")

# Weighted Sum Sij = Summation(Cij * uj|i)

weighted_predictions_round1 = tf.multiply(routing_weights_round1, secondary_capsule_predicted, name="weighted_predictions_round1")
weighted_sum_round1 = tf.reduce_sum(weighted_predictions_round1, axis=1, keepdims=True, name="weighted_sum_round1")

# vj = SQUASH(Sj) 

secondary_capsule_output_round1 = squash(weighted_sum_round1, ax=-2, name="secondary_capsule_output_round1")

secondary_capsule_output_round1 

# Update bias bij = bij + uj|i . vj

secondary_capsule_output_round1_tiled = tf.tile(secondary_capsule_output_round1, [1, 1152, 1, 1, 1], name="secondary_capsule_output_round1_tiled")

agreement_after_round1 = tf.matmul(secondary_capsule_predicted, secondary_capsule_output_round1_tiled, transpose_a=True, name="agreement_after_round1")

raw_weights_round2 = tf.add(raw_weights_round1, agreement_after_round1, name="raw_weights_round2")

# Round 2 

# Coupling Coeffecient Cij = softmax(bij) 

routing_weights_round2 = tf.nn.softmax(raw_weights_round2, axis=2, name="routing_weights_round2")

# Weighted Sum Sij = Summation(Cij * uj|i)

weighted_predictions_round2 = tf.multiply(routing_weights_round2, secondary_capsule_predicted, name="weighted_predictions_round2")
weighted_sum_round2 = tf.reduce_sum(weighted_predictions_round2, axis=1, keepdims=True, name="weighted_sum_round2")

# vj = SQUASH(Sj) 

secondary_capsule_output_round2 = squash(weighted_sum_round2, ax=-2, name="secondary_capsule_output_round2")

# Update bias bij = bij + uj|i . vj

secondary_capsule_output_round2_tiled = tf.tile(secondary_capsule_output_round2, [1, 1152, 1, 1, 1], name="secondary_capsule_output_round2_tiled")

agreement_after_round2 = tf.matmul(secondary_capsule_predicted, secondary_capsule_output_round2_tiled, transpose_a=True, name="agreement_after_round2")
raw_weights_round3 = tf.add(raw_weights_round2, agreement_after_round2, name="raw_weights_round3")

# Round 3 

# Coupling Coeffecient Cij = softmax(bij) 

routing_weights_round3 = tf.nn.softmax(raw_weights_round3, axis=2, name="routing_weights_round3")

# Weighted Sum Sij = Summation(Cij * uj|i)

weighted_predictions_round3 = tf.multiply(routing_weights_round3, secondary_capsule_predicted, name="weighted_predictions_round3")
weighted_sum_round3 = tf.reduce_sum(weighted_predictions_round3, axis=1, keepdims=True, name="weighted_sum_round3")

# vj = SQAUSH(sj)

secondary_capsule_output_round3 = squash(weighted_sum_round3, ax=-2, name="secondary_capsule_output_round3")

# Update bias bij = bij + uj|i . vj (OPTIONAL SINCE THIS IS THE LAST ROUND)

seecondary_capsule_output_round3_tiled = tf.tile(secondary_capsule_output_round2, [1, 1152, 1, 1, 1], name="secondary_capsule_output_round2_tiled")

agreement_after_round3 = tf.matmul(secondary_capsule_predicted, seecondary_capsule_output_round3_tiled, transpose_a=True, name="agreement_after_round3")

raw_weights_round4 = tf.add(raw_weights_round3, agreement_after_round3, name="raw_weights_round4")

# End of Dynamic Routing 


In [14]:
secondary_capsule_output = secondary_capsule_output_round3

In [16]:
# Estimated Class Probabilities 

def safe_norm(s, ax=-1, epsilon=1e-7, keepdims_v=False, name=None):
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=ax,
                                     keepdims=keepdims_v)
        return tf.sqrt(squared_norm + epsilon)

y_prob = safe_norm(secondary_capsule_output, ax=-2, name="y_prob")
y_prob_argmax = tf.argmax(y_prob, axis=2, name="y_prob_argmax") 

y_predicted = tf.squeeze(y_prob_argmax, axis=[1, 2], name="y_predicted")
